# Pyscreener quickstart

## Setting up a virtual screen

The object model of `pyscreener` is centered around the `VirtualScreen` object. Abstractly, this object represents a **unfiorm** screening protocol against which 10s to millions of compounds will be screened. Roughly, a virtual screening protocol includes the following information:
1. the target
2. the docking box
3. the screening parameters

The last of these is the most abstract, as we use this to lump all program-specific arguments together. For example, AutoDock Vina type programs might support only a few arguments like `--exhaustiveness` to control the exhaustiveness of the search process, but DOCK6 includes a variety of parameters to fine tune the search process, like `simplex_tors_step`. Obviously, both programs use some knowledge of the target (1) and the docking box (2), but beyond that, they don't have much in common in terms of how they're run.

There are a wealth of possble options for how a virtual screen *might* be conducted, but our primary goal with `pyscreener` is to support users who really want to rely on the base docking parameters to quickly filter out possible compound ideas with the *ability* to support as much customization as the raw docking programs allow. With an eye towards servicing that first group of users, our goal is to minimize the overhead needed to setup a virtual screen.

The following cell shows an example of setting up a screening protocol against the D4 dopamine receptor 5WIU using AutoDock Vina using the docking box centered at the given coordinates with the specified x-, y-, and z-radii. Because AutoDock Vina can leverage multiple cores during a docking run, we additionally specify that each process will be allotted `6` cores.

In [ ]:
import pyscreener as ps

md = ps.build_metadata("vina")
vs = ps.virtual_screen(
    "vina",
    receptors=["integration-tests/inputs/5WIU.pdb"], 
    center=(-18.2, 14.4, -16.1),
    size=(15.4, 13.9, 14.5),
    metadata_template=md,
    ncpu=6
)

To actually subject compounds to our virtual screening protocol, we simply `__call__()` our `VirtualScreen` on the SMILES string(s) of our compound(s) of interest

In [ ]:
scores = vs("c1ccccc1")
scores

Note that it's also possible to pass in multiple SMILES strings

In [ ]:
scores = vs("c1ccccc1", "O=C(Cc1ccccc1)NC1C(=O)N2C1SC(C2C(=O)O)(C)C")

We also support directly passing in a `List` of SMILES strings if you want to evaluate batches of compounds at a time

In [ ]:
smis = [
    "c1ccccc1",
   "O=C(Cc1ccccc1)NC1C(=O)N2C1SC(C2C(=O)O)(C)C",
   "C=CCN1CCC23C4C(=O)CCC2(C1CC5=C3C(=C(C=C5)O)O4)O"
]
scores = vs(smis)
scores

As mentioned above, we can customize the parameters of our virtual screen, such as the exhaustiveness. Most flags for AutoDock Vina are supported in this manner, simply pass in a dictionary to the `build_metadata()` function using the flag name as the key and the desired value

With a higher exhaustiveness, individual docking runs will take longer (time scales roughly linearly with exhaustiveness,) but our results should be more repeatable.

In [ ]:
md = ps.build_metadata("vina", dict(exhaustivness=32))
vs = ps.virtual_screen(
    "vina",
    receptors=["integration-tests/inputs/5WIU.pdb"], 
    center=(-18.2, 14.4, -16.1),
    size=(15.4, 13.9, 14.5),
    metadata_template=md,
    ncpu=6
)
scores = vs(smis)
scores

`pyscreener` also supports other programs in the AutoDock Vina family: QVina, Smina, and PSOVina. Simply pass in the desired program as the value to the `software` key in the `metadata` dictionary, like the cell below

In [ ]:
md = ps.build_metadata("vina", dict(software="qvina"))
vs = ps.virtual_screen(
    "vina",
    receptors=["integration-tests/inputs/5WIU.pdb"], 
    center=(-18.2, 14.4, -16.1),
    size=(15.4, 13.9, 14.5),
    metadata_template=md,
    ncpu=6
)
scores = vs(smis)
scores

Lastly, we also support DOCKing in `pyscreener`. Pass in `"dock"` for the `screen_type` argument to `build_metadata()` to accomplish this. *Note:* DOCK6 has a plethora of options with which to customize a DOCKing preparation **and** simulation. Some of these values are not supported at this time, but will be added in a future release of `pyscreener`!

In [ ]:
metadata = ps.build_metadata("dock")
vs = ps.virtual_screen(
    "vina",
    receptors=["integration-tests/inputs/5WIU.pdb"], 
    center=(-18.2, 14.4, -16.1),
    size=(15.4, 13.9, 14.5),
    metadata_template=md,
    ncpu=6
)
scores = vs(smis)
scores

## Distributing a Virtual Screen

Sometimes you might want to screen a *large* number of compounds at once and will have access to the resources over which to distribute these docking runs. `pyscreener` offers first class support for task distribution using `ray` in the backend. To change your code from a local to distributed setup, simply connect to the proper `ray` cluster before running your virtual screen, like so:

In [ ]:
import ray

ray.init(address=None, _redis_password=None)